#Labo 4 : Classifieurs Linéaires et Descente de Gradient
# 15 octobre 2024

In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

np.set_printoptions(precision=2)

# on définit un noyau pour rendre nos résultats déterministes
# (pour que tout le monde ait les mêmes resultats.)
np.random.seed(2)

# Fonctions de perte et gradients

Nous considérons ici des données labellisées $(x,y)$, $x$ est une entrée de dimension $d$ et $y\in\{-1,1\}$. Notre dataset contient n de ces paires. L'objectif de cette séance sera d'entraîner des classifieurs linéaires sur ce dataset. Ces classifieurs linéaires font la prédiction:
$$f( x ; w, b) = \text{sign}(w^T x + b) = 1 \quad \text{si}\quad w^T x + b\geq 0 \quad\text{et}\quad -1 \quad\text{sinon.}$$

Afin de simplifier le code et la notation, nous pouvons nous débarasser du terme du biais $b$. Pour ce faire, nous concatènons un $1$ à chaque vecteur $x$ de façon à ce que $x' = (x, 1) \in \mathbb{R}^{d+1}$ puis nous concatènons $b$ à $w$ de sorte que $w' = (w, b)$. Ainsi, $w^T x + b = w'^T x'$ et nous pouvons tout écrire en terme de transformations linéaires à la place de transformations affines. Nous pouvons maintenant omettre le terme de biais.

L'erreur d'entraînement est définie comme étant la moyenne des erreurs obtenues dans le dataset:

$$\text{Error}(w) = \frac{1}{n}\sum_{i=1}^n \mathbb{1}\{y_i \neq f(x_i ; w) \}$$

où $\mathbb{1}\{A\}$ est la fonction indicatrice.

**Exercice:** Nous ne pouvons pas directement minimiser cette fonction. *Pourquoi ?*

Puisque nous ne pouvons pas minimiser cette expression, nous allons plutôt minimiser 4 différentes fonctions de perte qui sont typiquement utilisées avec des modèles linéaires. Ces fonctions sont appelées des fonctions convexes de substitution. Elles sont convexes et possède un (sous-)gradient donc nous pouvons les optimiser avec la descente du gradient.

* régression linéaire $g(w; x, y) = \frac{1}{2}(x^T w - y)^2$.
* perceptron $g(w; x, y) = \max(0, -y x^T w)$.
* SVM $g(w; x, y) = \max(0, 1 - y x^T w)$.
* régression logistique $g(w; x, y) = \log( 1 + e^{- y x^T w})$.

La fonction objectif que nous allons minimiser est la moyenne des pertes de chaque exemple additionné à une régularisation de type $\ell^2$ d'hyperparamètre $\lambda$:

$$L(w) = \frac{1}{n}\sum_{i=1}^n g(w; x_i, y_i) + \frac{\lambda}{2} \| w\|^2 $$

Notez que pour le SVM, le terme de régularisation sert en fait à trouver la marge maximale de la frontière de décision.

**Exercice:** Complétez les fonctions suivantes qui effectuent le graphique de ces pertes en fonction de $z = x^T w$ en assumant que $y = 1$. Complétez également la fonction `error_counter` qui représente $\mathbb{1}\{y \neq f(x ; w) \} = \mathbb{1}\{ x^T w <0 \} = \mathbb{1}\{ z <0 \} $.


In [ ]:
# Question 1

def error_counter(z):
    # écrivez votre code ici
    return

def linearloss(z):
    # écrivez votre code ici
    return

def logisticloss(z):
    # écrivez votre code ici
    return

def perceptronloss(z):
    # écrivez votre code ici
    return

def svmloss(z):
    # écrivez votre code ici
    return

zz = np.linspace(-3,3,1000)
plt.figure()
for loss in [error_counter, linearloss, logisticloss,perceptronloss, svmloss]:
    plt.plot(zz, loss(zz), label=loss.__name__)

plt.ylim(-.5,4.5)
plt.legend()
plt.grid()
plt.show()

**Exercice:** Dérivez le gradient par rapport à $w$ pour chacune des fonctions de pertes définies ci-dessus.

**Exercice bonus:** Toutes ces fonctions de pertes ont un gradient qui est définit comme étant un scalaire multiplié pas $x$. Encore une fois, faites un graphique de ces scalaires en assumant que $y=1$.

## 1. Prétraitement de données

Pour notre problème de classification binaire, nous allons utiliser uniquement deux classes du dataset iris, soit la classe 1 et la classe 2. Ainsi, seulement les exemples appartenant à la classe 1 ou 2 seront considérés et par mesure de simplicité, nous allons changer leurs étiquettes à 1 et -1. Nous allons de plus utiliser seulement que deux caractéristiques (`feature`) des fleurs du dataset iris pour chaque exemple. Ceci permet de facilement visualiser les frontières de décisions.

Voici le code pour le pré-traitement:

In [ ]:
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    iris = np.loadtxt('http://www.iro.umontreal.ca/~dift3395/files/iris.txt')
else:
    iris = np.loadtxt('iris.txt')

def preprocess(data, label_subset, feature_subset, n_train):

    """Effectue une partition aléatoire des données en sous-ensembles
    train set et test set avec le sous-ensemble de classes label_subset
    et le sous-ensemble de feature feature_subset
    """
    # on extrait seulement les classes de label_subset
    data = data[np.isin(data[:,-1],label_subset),:]


    # on transforme les classes pour qu'elles soient [-1, 1]
    if len(label_subset)!=2:
        raise UserWarning('We are exclusively  dealing with binary classification.')
    data[data[:,-1]==label_subset[0],-1] = -1
    data[data[:,-1]==label_subset[1],-1] = 1

    # on extrait les features et leurs étiquettes
    data = data[:, feature_subset + [-1]]

    # on ajoute une colonne pour le biais
    data = np.insert(data, -1, 1, axis=1)

    # on sépare en train et test
    inds = np.arange(data.shape[0])
    np.random.shuffle(inds)
    train_inds = inds[:n_train]
    test_inds = inds[n_train:]
    trainset = data[train_inds]
    testset = data[test_inds]

    # on normalise les données pour qu'elles soient de moyenne 0
    # et d'écart-type 1 par caractéristique et on applique
    # ces mêmes transformations au test set
    mu = trainset[:,:2].mean(axis=0)
    sigma  = trainset[:,:2].std(axis=0)
    trainset[:,:2] = (trainset[:,:2] -mu)/sigma
    testset[:,:2] = (testset[:,:2] -mu)/sigma

    return trainset, testset


trainset, testset = preprocess(iris, label_subset=[1,2], feature_subset=[2,3], n_train=75)

## 2. Quelques fonctions utilitaires

In [ ]:
def scatter(theset, marker='o'):
    d1 = theset[theset[:, -1] > 0]
    d2 = theset[theset[:, -1] < 0]
    plt.scatter(d1[:, 0], d1[:, 1], c='b', marker=marker, label='class 1', alpha=.7)
    plt.scatter(d2[:, 0], d2[:, 1], c='g', marker=marker, label='class 0', alpha=.7)
    plt.xlabel('x_0')
    plt.ylabel('x_1')


def finalize_plot(title):
    plt.title(title)
    plt.grid()
    plt.legend()

scatter(trainset, marker='x')
scatter(testset, marker='^')
finalize_plot('train and test data')

Nous représentons et définissons ci-dessous la frontière de décision des modèles linéaires que nous allons entraîner. Comme la règle de décision pour faire la prédiction d'un exemple est commune à tous ces modèles, la frontière de décision est alors calculée de la même façon, c'est-à-dire en isolant $x_1$ dans $f(x) = x^{T}w = 0$.

In [ ]:
def decision_boundary(w):
    if w[1]==0:
        raise RuntimeWarning("This decision boundary is either vertical, either undefined.")

    # hack to avoid changing the boundaries
    # astuce pour éviter de changer les limites
    xlim = plt.xlim()
    ylim = plt.ylim()

    xx = np.linspace(-10, 10, 2)
    yy = -(w[2] + w[0]*xx)/w[1]
    plt.plot(xx, yy, c='r', lw=2, label='f(x)=0')

    # hack to avoid changing the boundaries
    # astuce pour éviter de changer les limites
    plt.xlim(xlim)
    plt.ylim(ylim)

w0 = np.array([1,-1,1])
scatter(trainset)
decision_boundary(w0)
finalize_plot('A random classifier')

## 3. Classe Parent

Nous définissons ici une classe parent qui sera passée par la suite dans les classes enfants qui sont présentées ci-dessous. En appellant la fonction `super().__init__` à l'intérieur des classes enfants, nous pouvons ainsi hériter d'attributs de la classe parent sans devoir les implémenter à nouveau. Nous avons une classe enfant pour chacun des classifieurs linéaires et dans ces classes, votre rôle sera de complétez les méthodes `loss` et `gradient`.

#### La Descente de Gradient
Nous voulons minimiser la fonction de perte $L$. Cette fonction est différentiable, donc nous pouvons utiliser l'algorithme de descente de gradient: commencer par n'importe quelle initialisation du paramètre $w_0$ et répéter pour $t\in\{0, \dots, t_\max \}$:
$$w_{t+1} = w_t - \eta \nabla L (w_t) \; .$$
Sous certaines conditions sur la taille du pas $\eta$ et de la perte $L$, cet algorithme est garanti de converger à un minimum de $L$. Vous devez compléter cet algorithme dans cette classe parent.

#### Fonction de prédiction
Vous devrez remplir la fonction `predict` de cette classe. Cette fonction doit retourner les prédictions étant donnée une matrice de données `X` (de taille $n_{exemples} \times n_{features}$).


#### Fonction de test
Vous devrez remplir la fonction `error_rate` de cette classe. Cette fonction prend en entrée une matrice de données `X` (de taille $n_{exemples} \times n_{features}$) et les étiquettes correspondantes (`y`, vecteur de taille $n_{examples}$), et retourne l'erreur moyenne du classifieur sur ces données-là.

#### Fonction d'entraînement
Vous devrez écrire le code de la boucle for de la fonction `train`. Chaque étape de cette boucle doit effectuer une mise-à-jour du paramètre $w$ avec la méthode du gradient, et mettre-à-jour les listes `losses` et `errors`.

In [ ]:
from IPython.display import clear_output

class LinearModel:
    """"Classe parent pour tous les modèles linéaires.
    """

    def __init__(self, w0, reg):
        """Les poids et les biais sont définis dans w.
        L'hyperparamètre de régularisation est reg.
        """
        self.w = np.array(w0, dtype=float)
        self.reg = reg

    def predict(self, X):
        """Retourne f(x) pour un batch X
        """
        # écrivez votre code ici
        return np.zeros(X.shape[0])

    def error_rate(self, X, y):
        """Retourne le taux d'erreur pour un batch X
        """
        # écrivez votre code ici
        return 0.

    # les méthodes loss et gradient seront redéfinies dans les classes enfants
    def loss(self, X, y):
        return 0

    def gradient(self, X, y):
        return self.w

    def train(self, data, stepsize, n_steps, plot=False):
        """Faire la descente du gradient avec batch complet pour n_steps itération
        et un taux d'apprentissage fixe. Retourne les tableaux de loss et de
        taux d'erreur vu apres chaque iteration.
        """

        X = data[:,:-1]
        y = data[:,-1]
        losses = []
        errors = []

        for i in range(n_steps):
            # Gradient Descent
            # WRITE CODE HERE

            # Update losses
            # WRITE CODE HERE]

            # Update errors
            # WRITE CODE HERE

            # Plot
            if plot and i % 2 == 0:
              clear_output(wait=True)
              plt.figure()
              scatter(trainset, marker='o')
              scatter(testset, marker='x')
              decision_boundary(self.w)
              finalize_plot(i)
              plt.show()

        print("Training completed: the train error is {:.2f}%".format(errors[-1]*100))
        return np.array(losses), np.array(errors)

def test_model(modelclass, w0=[-3.0, 3.0, 0.1], reg=.1, stepsize=.2, plot=False):
    """Crée une instance de modelclass, entraîne la, calcule le taux d'erreurs sur un
    test set, trace les courbes d'apprentissage et la frontieres de decision.
    """
    model = modelclass(w0, reg)
    training_loss, training_error = model.train(trainset, stepsize, 100, plot=plot)
    print("The test error is {:.2f}%".format(
      model.error_rate(testset[:,:-1], testset[:,-1])*100))
    print('Initial weights: ', w0)
    print('Final weights: ', model.w)

    # learning curves
    fig, (ax0, ax1) = plt.subplots(ncols=2, figsize=(8,2))
    ax0.plot(training_loss)
    ax0.set_title('loss')
    ax1.plot(training_error)
    ax1.set_title('error rate')

    # data plot
    plt.figure()
    scatter(trainset, marker='x')
    scatter(testset, marker='^')
    decision_boundary(model.w)
    finalize_plot(modelclass.__name__)

test_model(LinearModel)

## 4. Linear Regression/Régression linéaire

**Exercice:** Complétez les méthodes `loss` et `gradient`.

Rappel la fonction de perte (loss) est donnée par  $$L(w) = \frac{1}{n}\sum_{i=1}^n g(w; x_i, y_i) + \frac{\lambda}{2} \| w\|^2 $$

In [ ]:
# Question 2

class LinearRegression(LinearModel):

    def __init__(self, w0, reg):
        super().__init__(w0, reg)

    def loss(self, X, y):
        """Calcule la perte moyenne pour une batch X.
        Prend en entrée une matrice X et le vecteur y et retourne un scalaire.
        """
        # écrivez votre code ici
        pass

    def gradient(self, X, y):
        """Calcule le gradient de la fonction de perte par rapport à w pour un batch X.
        Prend en entrée une matrice X et le vecteur y.
        Retourne un vecteur de la meme taille que w.
        """
        # écrivez votre code ici
        pass

test_model(LinearRegression, plot=False)

## 5. Perceptron

**Exercice:** Complétez les méthodes `loss` et `gradient`.

In [ ]:
# Question 3

class Perceptron(LinearModel):

    def __init__(self, w0, reg):
        super().__init__(w0, reg)

    def loss(self, X, y):
        # write code here/écrivez votre code ici
        pass

    def gradient(self, X, y):
        # write code here/écrivez votre code ici
        pass


test_model(Perceptron, reg=0, stepsize=1)

## 6. SVM

**Exercice:** Complétez les méthodes `loss` et `gradient`.

In [ ]:
# Question 4

class SVM(LinearModel):

    def __init__(self, w0, reg):
        super().__init__(w0, reg)

    def loss(self, X, y):
        # write code here/écrivez votre code ici
        pass

    def gradient(self, X, y):
        # write code here/écrivez votre code ici
        pass


test_model(SVM, reg=.1, stepsize=1)

## 7. Logistic Regression/Régression logistique

**Exercice:** Complétez les méthodes `loss` et `gradient`.

In [ ]:
# Question 5

class LogisticRegression(LinearModel):

    def __init__(self, w0, reg):
        super().__init__(w0, reg)

    def loss(self, X, y):
        # write code here/écrivez votre code ici
        pass

    def gradient(self, X, y):
        # write code here/écrivez votre code ici
        pass


test_model(LogisticRegression)

## Conclusion

C'est terminé! Vous avez maintenant réussi à entraîner 4 modèles à l'aide de fonctions de pertes les plus connues. Pour être exacte, la fonction de perte du perceptron n'est plus vraiment utilisée. Savez-vous pourquoi? Êtes-vous en mesure de comparer le comportement de la perte du SVM versus celle du perceptron lorsque vous jouez avec différents hyperparamètres et initialisations? Que pouvez-vous dire du comportement de la frontière de décision du perceptron?

Actuellement, les jeux de données en apprentissage machine sont tellement énormes que nous utilisons la descente du gradient stochastique plutôt que la descente du gradient complète. Vous pouvez d'ailleurs essayer de l'implémenter dans la classe parent `LinearModel`.

Vous pouvez également explorer différents hyperparamètres et observer vos résultats!
